# PHASE 1) DATA ACQUISITON 

In [27]:
import bs4 as bs
import datetime as dt # use datetime to specify dates for the Pandas datareade
import os
import pandas as pd
import pandas_datareader.data as web
#from pandas_datareader import data as pdr #new
import pickle
import requests
#import fix_yahoo_finance #new

## Step 1 : web scrapping in wiki to get tickers

In [2]:
# PHASE A) To get the tickers of the 100 companies

def save_nasdaq100_tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/NASDAQ-100')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[1].text
        tickers.append(ticker.rstrip())
        
    with open("nasdaq100tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers

#save_nasdaq100_tickers()

## Step 2: Pull Data from Yahoo Finance

In [45]:
def get_data_from_yahoo(reload_nasdaq100=False):
    if reload_nasdaq100:
        tickers = save_nasdaq100_tickers()
    else:
        with open("nasdaq100tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')

    start = dt.datetime(2010, 1, 1)
    end = dt.datetime.now()
    for ticker in tickers:
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = web.DataReader(ticker, 'yahoo', start, end)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))

get_data_from_yahoo()

## Step 3: Join 100 companies in one

In [46]:
# To join all the tickers csv files in one , taking just the colum Adj Close which is the one what interested us to perform the 
# analysis 

def compile_data():
    with open("nasdaq100tickers.pickle", "rb") as f:
        tickers = pickle.load(f)

    main_df = pd.DataFrame()

    for count, ticker in enumerate(tickers):
        df = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
        df.set_index('Date', inplace=True)

        df.rename(columns={'Adj Close': ticker}, inplace=True)
        df.drop(['Open', 'High', 'Low', 'Close', 'Volume'], 1, inplace=True)

        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how='outer')

        if count % 10 == 0:
            print(count)
    print(main_df.head())
    main_df.to_csv('nasdaq100_joined_closes.csv')


compile_data()

0
10
20
30
40
50
60
70
80
90
100
                 ATVI       ADBE   AMD       ALXN   ALGN       GOOGL  \
Date                                                                   
2010-01-04  10.193225  37.090000  9.70  24.125000  18.50  313.688690   
2010-01-05  10.211267  37.700001  9.71  23.780001  18.01  312.307312   
2010-01-06  10.157144  37.619999  9.57  23.840000  17.48  304.434448   
2010-01-07   9.913588  36.889999  9.47  23.930000  17.43  297.347351   
2010-01-08   9.832404  36.689999  9.43  23.975000  17.66  301.311310   

                  GOOG        AMZN       AAL       AMGN    ...           VRSN  \
Date                                                       ...                  
2010-01-04  311.349976  133.899994  4.557168  48.350658    ...      21.007998   
2010-01-05  309.978882  134.690002  5.073073  47.931801    ...      21.228777   
2010-01-06  302.164703  132.250000  4.862889  47.571617    ...      20.999506   
2010-01-07  295.130463  130.000000  5.006195  47.136017  